In [ ]:
import json
from apps.emojirama.models import Emojirama
from django.conf import settings

r = settings.REDIS
r.flushall()

In [ ]:
def write_keys_for_emojirama(emojirama):
    scene_list = e.board["scenes"].keys()
    p = r.pipeline()
    for scene in scene_list:
        for i, row in enumerate(e.board["scenes"][scene]["data"]):
            for j, square in enumerate(row):
                p.hset(f"{emojirama.id}___{scene}___{j}___{i}___square", 'emoji', square['emoji'])
                p.hset(f"{emojirama.id}___{scene}___{j}___{i}___square", 'color', square['color'])
                p.hset(f"{emojirama.id}___{scene}___{j}___{i}___square", 'tone', square['tone'])
                p.hset(f"{emojirama.id}___{scene}___{j}___{i}___square", 'position', str(square['position']))
    return p.execute()

In [ ]:
e = Emojirama.objects.all().first()
scene_list = e.board["scenes"].keys()
pipeline_results = write_keys_for_emojirama(e)

In [ ]:
len(pipeline_results)

In [ ]:
def get_json_from_server_for_emojirama(emojirama):
    emojirama_id = emojirama.id
    
    # to round trips to redis
    keys = r.keys(f'{emojirama_id}___*___*___*___square')
    p = r.pipeline()
    for key in keys:
        p.hgetall(key)
    hashes = p.execute()
    key_hash_dict = {key: _hash for key, _hash in zip(keys,hashes)}
    
    
    scene_list = emojirama.board["scenes"].keys()
    json_data = {"scenes":{scene: {} for scene in scene_list}}
    for scene in scene_list:
        row_num = 0
        col_num = 0
        rows = []

        while True:
            row = []
            col_num = 0
            try:
                _ = key_hash_dict[f"{emojirama_id}___{scene}___{row_num}___{col_num}___square"]
            except KeyError:
                break
            while True:
                try:
                    h = key_hash_dict[f"{emojirama_id}___{scene}___{row_num}___{col_num}___square"]
                    h["tone"] = int(h["tone"])
                    h["position"] = json.loads(h["position"])
                    row.append(h)
                except KeyError:
                    break
                col_num += 1
                
            rows.append(row)
            
            row_num += 1
            
        json_data["scenes"][scene]["data"] = rows
                
            
    return json_data


In [ ]:
a = get_json_from_server_for_emojirama(e)

In [ ]:
a